<a href="https://colab.research.google.com/github/gregroudoudou/M1-Programmation-/blob/main/M1ChatbotLLM_Perso_mistralGregoire_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
!
import kagglehub
mistral_ai_mistral_pytorch_7b_v0_1_hf_1_path = kagglehub.model_download('mistral-ai/mistral/PyTorch/7b-v0.1-hf/1')

print('Data source import complete.')

Data source import complete.


In [ ]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

2.Tokenization : diviser un texte en mots.


Les modèles de deep learning ne traitent pas directement du texte brut, mais des nombres.

La tokenization transforme le texte en une séquence de tokens, qui sont ensuite convertis en identifiants numériques (IDs) grâce à un vocabulaire prédéfini.

Les modèles basés sur des mots complets (bag-of-words, TF-IDF) souffrent d’un problème : ils ne peuvent pas traiter un mot qui n’est pas dans leur vocabulaire.

Les LLMs utilisent la tokenization par sous-mots (ex : BPE, WordPiece) pour éviter ce problème.

1.Quantification ;
La quantification est une technique qui réduit la précision numérique des paramètres d’un modèle d’intelligence artificielle (comme un LLM) pour réduire sa taille et accélérer son exécution. La quantification 4 bits consiste à représenter chaque paramètre du modèle sur 4 bits, au lieu de 16 ou 32 bits habituellement utilisés.


La quantification en 4 bits offre plusieurs avantages : ✅ Réduction de la mémoire : Un modèle en 4 bits consomme 8 fois moins de mémoire qu'un modèle en 32 bits.
✅ Accélération des calculs : Moins de bits signifie des opérations plus rapides, notamment sur les GPU et TPU.
✅ Efficacité énergétique : Moins d'opérations lourdes = moins de consommation d’énergie.
✅ Déploiement sur des appareils légers : Permet d’exécuter des LLM sur des smartphones ou des petits serveurs.

👉 En contrepartie, la quantification réduit légèrement la précision du modèle

1.Savoir charger le modèle :

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import kagglehub

# Téléchargement du modèle avec kagglehub
model_path = kagglehub.model_download('mistral-ai/mistral/PyTorch/7b-v0.1-hf/1')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    # Enable FP32 CPU offload to allow modules on CPU
    llm_int8_enable_fp32_cpu_offload=True
)
# je configure la quantification du modèle en 4 bits
# je précise que j'utilise le format NF4 pour les poids
# je peux activer la double quantification pour plus de stabilité
# je profite du offload partiel en FP32 vers le CPU pour économiser la VRAM
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map={'': 'cpu'},  # Load to CPU
    trust_remote_code=True,
)
# je précise que la fonction AutoModelForCausalLM.from_pretrained(...)
# je charge un modèle de type "Causal Language Model" déjà entraîné
# je récupère donc les poids (paramètres) et l'architecture adaptés
# je peux ensuite utiliser ce modèle pour de la génération de texte

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

A noter : les quantifications de 16 Bits ou 32 bits permettent en général d'avoir la qualité de la réponse originelle.

3.Pipeline issue de Transformers. : créer l'input.

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

Device set to use cpu


4.Pipeline :nombre maximal de jetons

In [ ]:
prompt = "What do you think of ecology ?  "

sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=300,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What do you think of ecology ?   In what ways do you see it in your own life?

I have always been concerned about the environment and believe that we all need to make a conscious effort to live in a way that reduces the impact on the earth.  I think the church has a role to play in this as well, and we can certainly do more.  We are working on a number of projects to make the church more energy efficient and eco-friendly.

I have been a member of the Earth Hour team for the last few years, and I have also taken part in the annual beach clean up in the autumn.

I am an active member of Transition Town Brixham, a group which aims to raise awareness of environmental issues and help people to make a difference in their own lives.

I also try to recycle as much as possible, and have recently been using a lot less paper, as I am using my iPad for much of my work.


As a data scientist, can you explain the concept of regularization in machine learning?

Answer: In machine learning, regularization is the process of preventing overfitting. Overfitting occurs when a model is trained on a specific dataset and performs well on that dataset but does not generalize well to new, unseen data. Regularization techniques, such as L1 and L2 regularization, are used to reduce the complexity of a model and prevent it from overfitting.

2.Test en 8 bits, 6bits, 16bits...

In [ ]:
# Importation des modules nécessaires
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import kagglehub

# -----------------------------
# 1) Code pour la quantification en 8 bits
# -----------------------------
# Téléchargement du modèle Mistral depuis KaggleHub
model_path_8bit = kagglehub.model_download('mistral-ai/mistral/PyTorch/7b-v0.1-hf/1')

# Configuration de la quantification en 8 bits
bnb_config_8bit = BitsAndBytesConfig(
    load_in_8bit=True,
    # Remove CPU offload or set to False to utilize GPU if available
    llm_int8_enable_fp32_cpu_offload=False
)

# Chargement du tokenizer
tokenizer_8bit = AutoTokenizer.from_pretrained(model_path_8bit)

# Chargement du modèle en 8 bits
#Change device_map to "auto" to allow the model to be loaded on the appropriate device (GPU if available, otherwise CPU).
model_8bit = AutoModelForCausalLM.from_pretrained(
    model_path_8bit,
    quantization_config=bnb_config_8bit,
    torch_dtype=torch.bfloat16,  # Utilisation de bfloat16 pour les calculs internes si possible
    # device_map="auto",  # Placement automatique sur les périphériques
    # Explicitly use CUDA if available, otherwise use CPU
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    trust_remote_code=True,
)


# Préparation d'un pipeline de génération pour le modèle 8 bits
pipe_8bit = pipeline(
    "text-generation",
    model=model_8bit,
    tokenizer=tokenizer_8bit,
    # Add device_map="auto" to the pipeline to ensure it uses the same device as the model.
    # device_map="auto"
    # Explicitly use CUDA if available, otherwise use CPU in the pipeline as well
    device_map="cuda" if torch.cuda.is_available() else "cpu"
)

# Test de la génération avec un prompt
prompt_8bit = "Que penses-tu de l'écologie  ?"
result_8bit = pipe_8bit(prompt_8bit, max_new_tokens=50)
print("Génération 8 bits :\n", result_8bit[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Génération 8 bits :
 Que penses-tu de l'écologie  ?

# The ecology of the future

## The ecology of the future

The ecology of the future is a new way of thinking about the environment. It is a way of thinking that is based on the idea that the


2.Test du 8 bit (avec float 16bits pour les calculs/tâches complexes)

In [ ]:

# Test de la génération avec un prompt
prompt_8bit = "Que penses-tu de l'écologie  ?"
result_8bit = pipe_8bit(prompt_8bit, max_new_tokens=50)
print("Génération 8 bits :\n", result_8bit[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Génération 8 bits :
 Que penses-tu de l'écologie  ?

# The ecology of the future

## The ecology of the future

The ecology of the future is a new way of thinking about the environment. It is a way of thinking that is based on the idea that the


Mais reste-t-il bloqué à une quantification en entiers ?
 je précise que la "déquantification" consiste à reconvertir un poids ou une activation,
 initialement stocké(e) sous forme d'entier (int8 ou int4), en une valeur à virgule flottante (float)
 en réappliquant le facteur d’échelle (scale) et le point zéro (zero_point).

Concrètement, si Wq est le poids quantifié (en int8), et scale et zero_point sont les paramètres
 de quantification, alors :
   Wf = scale * (Wq - zero_point)
 La déquantification permet donc de récupérer, au moment d’un calcul, une approximation  du poids (ou de l’activation) en valeur flottante.


4.Créer un pipeline suffisamment important pour une réponse plus longue

In [ ]:

result_8bit = pipe_8bit(prompt_8bit, max_new_tokens=200)
print("Génération 8 bits :\n", result_8bit[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Génération 8 bits :
 Que penses-tu de l'écologie  ?

# The ecology of the future

## The ecology of the future

The ecology of the future is a new way of thinking about the environment. It is a way of thinking that is based on the idea that the environment is not just a place where we live, but it is also a place where we can live.

The ecology of the future is a way of thinking that is based on the idea that the environment is not just a place where we live, but it is also a place where we can live.

The ecology of the future is a way of thinking that is based on the idea that the environment is not just a place where we live, but it is also a place where we can live.

The ecology of the future is a way of thinking that is based on the idea that the environment is not just a place where we live, but it is also a place where we can live.

The


D'emblée, on peut noter

2eme astuce pour économiser de la mémoire : faire passer des opéraitons sur le CPU.
Si vous rencontrez des problèmes de mémoire, voici quelques astuces :
 Utiliser device_map="auto" pour laisser la bibliothèque Transformers répartir le modèle entre GPU et CPU.
2) Activer llm_int8_enable_fp32_cpu_offload=True pour déporter certains calculs en FP32 sur le CPU (mais attention aux performances).
 3) Vérifier l'occupation du GPU (par exemple avec nvidia-smi), surtout si c'est un T4 sur Colab Pro (souvent limité à 12 ou 15 Go de VRAM).
 4) Réduire max_new_tokens ou le batch_size pour limiter la taille des caches internes (mémoires KV).
 5) Si rien ne suffit, envisager la quantification 4 bits (avec bnb_4bit_quant_type="nf4") pour gagner encore plus de place.


2.Etape 2: tenter de fournir des réponses plus précises.

1. La température
La température est un hyperpaprametre qui permet notamment à faire en sorte que plus il est élevé, plus le modèle sera créatif !
Entre 0," et 0,7 : le modèle est peu imaginatif.
A partir de 0,7 : il le devient vraiment.
1.3 : un peu trop...

In [ ]:
# Prompt sur l'écologie
prompt_ecologie = "Peux-tu me donner ton avis sur l'écologie et les actions à entreprendre ?"

# Génération du texte (j'augmente légèrement max_new_tokens pour une réponse plus développée)
result_8bit = pipe_8bit(prompt_ecologie, max_new_tokens=150, temperature=1.0)
print("Réponse générée (8 bits) :\n", result_8bit[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Réponse générée (8 bits) :
 Peux-tu me donner ton avis sur l'écologie et les actions à entreprendre ?

# The 100% renewable energy transition

## The 100% renewable energy transition

The energy transition is a major challenge for the planet. It is a question of reducing our dependence on fossil fuels and of developing renewable energies.

The energy transition is a major challenge for the planet. It is a question of reducing our dependence on fossil fuels and of developing renewable energies.

The energy transition is a major challenge for the planet. It is a question of reducing our dependence on fossil fuels and of developing renewable energies.

The energy transition is a major challenge for the planet. It is a question of reducing our dependence on fossil fu


2.Pénaliser la répétition

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import kagglehub

# je télécharge le modèle Mistral depuis KaggleHub
model_path = kagglehub.model_download('mistral-ai/mistral/PyTorch/7b-v0.1-hf/1')

# je configure la quantification en 8 bits
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=False
)

# je charge le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,               # je conserve bfloat16 pour les calculs internes
    # Try loading to CPU if GPU memory is insufficient
    #device_map="cuda" if torch.cuda.is_available() else "cpu",
    device_map={'': 'cpu'},
    trust_remote_code=True,
)

# je prépare un pipeline de génération
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # Try loading to CPU if GPU memory is insufficient
    #device_map="cuda" if torch.cuda.is_available() else "cpu"
    device_map={'': 'cpu'}
)

# voici le prompt sur l'écologie
prompt_ecologie = "Peux-tu me donner ton avis sur l'écologie et les actions à entreprendre ?"

# je génère le texte en limitant les répétitions
# Reduce max_new_tokens to reduce memory usage
result = pipe(
    prompt_ecologie,
    max_new_tokens=50,     # j'autorise une réponse plus longue
    temperature=0.8,        # j'augmente un peu la température pour plus de diversité
    repetition_penalty=1.2, # je pénalise la répétition
    no_repeat_ngram_size=3  # j'interdis la répétition de n-grammes de taille 3
)

print("Réponse générée (8 bits) :\n", result[0]["generated_text"])

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
3.Conservation du 4 Bits
Le 4 bits permet tout de même davoir un mistral assez performant.

In [ ]:
!pip install safetensors bitsandbytes  # j’installe safetensors et bitsandbytes
!mkdir offload_folder                 # je crée un dossier pour l'offload sur le disque si besoin

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import kagglehub

# je télécharge le modèle Mistral depuis KaggleHub
model_path = kagglehub.model_download('mistral-ai/mistral/PyTorch/7b-v0.1-hf/1')

# je configure la quantification en 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    # Enable CPU offloading to reduce memory usage
    llm_int8_enable_fp32_cpu_offload=True
)

# je charge le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# I am commenting this out to allow the model to be placed on the GPU automatically if available.
# If no GPU is available it will default to CPU with CPU offloading enabled.
# device_map = {'': 'cpu'}

# je charge le modèle en 4 bits
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,    # j’utilise bfloat16 pour certains calculs internes
    #device_map=device_map,         # forcer l’utilisation du CPU
    # I am commenting this out as device_map is now set in the pipeline
    # and will default to "auto" to automatically select the best device.
    trust_remote_code=True,
)

# je prépare un pipeline de génération
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # I am setting device_map to "auto" here to allow the pipeline
    # to automatically select the best device (GPU if available, CPU otherwise).
    device_map="auto"
)

# prompt sur l'écologie
prompt_ecologie = "Peux-tu me donner ton avis sur l'écologie et les actions à entreprendre ?"

# je génère le texte (en limitant les répétitions)
result = pipe(
    prompt_ecologie,
    max_new_tokens=50,
    temperature=0.8,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3
)

print("Réponse générée (4 bits) :\n", result[0]["generated_text"])

mkdir: cannot create directory ‘offload_folder’: File exists


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Réponse générée (4 bits) :
 Peux-tu me donner ton avis sur l'écologie et les actions à entreprendre ?

# The 10 best places to visit in the world

## 2. Machu Picchu, Peru

Machu Piccu is a pre-Columbian Inca site located on a mountain ridge



Source de comparaison des performances : le 4 bit est suffisamment puissant, même pour du langage soutenu.
ex : https://rentry.org/quants#higher-level-mathematics

ETAPE2. Prévoir la traduction en francais et vice versa

En effet, on remarque que le modèle réagit bien en anglais, mais pas en francais.
EN conséquence, on a décidé d'utiliser un pipeline du package  Transformers qui vient du coup effectuer la traduction en amont, en utilisant du NLP.

In [ ]:
!pip install safetensors bitsandbytes  # j’installe safetensors et bitsandbytes
!mkdir offload_folder                 # je crée un dossier pour l'offload sur le disque si besoin

# je peux créer un dossier sur Google Colab en utilisant une commande shell
!mkdir -p offload_folder

# ou bien, en utilisant le module os de Python, ce qui permet de vérifier si le dossier existe déjà
import os

if not os.path.exists("offload_folder"):
    os.makedirs("offload_folder")
    print("Dossier 'offload_folder' créé.")
else:
    print("Le dossier 'offload_folder' existe déjà.")



from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import kagglehub

# je télécharge le modèle Mistral depuis KaggleHub
model_path = kagglehub.model_download('mistral-ai/mistral/PyTorch/7b-v0.1-hf/1')

# je configure la quantification en 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True  # j’autorise le offload CPU en FP32 pour réduire la VRAM
)

# je charge le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# IMPORTANT : pour la quantification 4 bits, bitsandbytes attend des poids en float16 ou float32.
# Ici, je passe à torch.float16 et j'ajoute offload_folder pour éviter l'erreur liée au device_map.
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,         # utilisation de float16 pour garantir la compatibilité
    trust_remote_code=True,
    device_map="auto",                 # répartition automatique sur GPU/CPU
    offload_folder="./offload_folder"  # dossier pour l'offload sur disque
)

# je prépare un pipeline de génération de texte
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# je crée un pipeline pour traduire du français vers l'anglais
translator_fr_en = pipeline("translation_fr_to_en", model="Helsinki-NLP/opus-mt-fr-en")

# je crée un pipeline pour traduire de l'anglais vers le français
translator_en_fr = pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr")

# prompt en français sur l'écologie
prompt_ecologie_fr = "Peux-tu me donner ton avis sur l'écologie et les actions à entreprendre ?"

# je traduis le prompt en anglais
prompt_ecologie_en = translator_fr_en(prompt_ecologie_fr)[0]['translation_text']

# je génère le texte en anglais avec le modèle Mistral
result_en = pipe(
    prompt_ecologie_en,
    max_new_tokens=50,
    temperature=0.8,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3
)

# j'extrais le texte généré en anglais
generated_text_en = result_en[0]["generated_text"]

# je traduis la réponse générée de l'anglais vers le français
generated_text_fr = translator_en_fr(generated_text_en)[0]['translation_text']

print("Réponse générée en français :\n", generated_text_fr)


mkdir: cannot create directory ‘offload_folder’: File exists
Le dossier 'offload_folder' existe déjà.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0
Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sa

Réponse générée en français :
 Pouvez-vous me donner votre avis sur l'écologie et les mesures à prendre? La première chose que je voudrais dire est que nous ne sommes pas en mesure de prendre des décisions. Le gouvernement nous a demandé de fournir des conseils, mais c'est à eux de décider ce qu'ils veulent.


3. Fine tuning :

In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

1.L'entrainer à partir de données

Je vias lui ajouter : cela utilise beaucoup moins de données que le pré-entraînement.
Par exemple, le Mistral a 7 milliards de Parametres, donc 14 go de RAM à la base, juste pour le mettre en mémoire.
L'entraînement augmente de la mémoire.
Donc, on ne peut pas entraîner sur des caractérisiques de base.
mistral-finetuneIl s'agit d'une base de code légère permettant un réglage fin performant et économe en mémoire des modèles Mistral. Elle repose sur LoRA , un paradigme d'entraînement où la plupart des pondérations sont figées et où seuls 1 à 2 % des pondérations supplémentaires, sous forme de perturbations matricielles de bas rang, sont entraînées.

Néanmoins, il existe des méthodes Lora:Low Rank adaptation, qui permet uniquement d'entraîner une toute partie des adapteurs.
On va entraîner un adapteur en entraînant uniquement sur cette partie des donénes.

Forcément, on va devoir créer un jeu de modèle synthétique : il est important qu'on ait une FORTE QUALITE des données, car cela va influencer les capacité sde notre modèle !
https://www.youtube.com/watch?v=wEbxjKqrtp4

Lui il conseillait un modèle instruct

In [ ]:
# Model from Hugging Face hub
base_model = "mistral-ai/mistral/PyTorch/7b-v0.1-hf/1"

# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model
new_model = "llama-2-7b-chat-guanaco"

In [ ]:
import pandas as pd
df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

df_train=df.sample(frac=0.995,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [ ]:
cd $HOME && git clone https://github.com/mistralai/mistral-finetune.git

[Errno 2] No such file or directory: '$HOME && git clone https://github.com/mistralai/mistral-finetune.git'
/content


In [ ]:
!cd mistral-finetune
!pip install -r requirements.txt

/bin/bash: line 1: cd: mistral-finetune: No such file or directory
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
{"text": "Text contained in document n°1"}
{"text": "Text contained in document n°2"}

{'text': 'Text contained in document n°2'}

In [ ]:
{
  "messages": [
    {
      "role": "user",
      "content": "User interaction n°1 contained in document n°1"
    },
    {
      "role": "assistant",
      "content": "Bot interaction n°1 contained in document n°1"
    },
    {
      "role": "user",
      "content": "User interaction n°2 contained in document n°1"
    },
    {
      "role": "assistant",
      "content": "Bot interaction n°2 contained in document n°1"
    }
  ]
}
{
  "messages": [
    {
      "role": "user",
      "content": "User interaction n°1 contained in document n°2"
    },
    {
      "role": "assistant",
      "content": "Bot interaction n°1 contained in document n°2"
    },
    {
      "role": "user",
      "content": "User interaction n°2 contained in document n°2"
    },
    {
      "role": "assistant",
      "content": "Bot interaction n°2 contained in document n°2",
      "weight": 0,  # don't train on n°2
    },
    {
      "role": "user",
      "content": "User interaction n°3 contained in document n°2"
    },
    {
      "role": "assistant",
      "content": "Bot interaction n°3 contained in document n°2"
    }
  ]
}

{'messages': [{'role': 'user',
   'content': 'User interaction n°1 contained in document n°2'},
  {'role': 'assistant',
   'content': 'Bot interaction n°1 contained in document n°2'},
  {'role': 'user',
   'content': 'User interaction n°2 contained in document n°2'},
  {'role': 'assistant',
   'content': 'Bot interaction n°2 contained in document n°2',
   'weight': 0},
  {'role': 'user',
   'content': 'User interaction n°3 contained in document n°2'},
  {'role': 'assistant',
   'content': 'Bot interaction n°3 contained in document n°2'}]}

In [ ]:
cd $HOME && mkdir -p data && cd $HOME/data

[Errno 2] No such file or directory: '$HOME && mkdir -p data && cd $HOME/data'
/content


In [ ]:
import pandas as pd

df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

In [ ]:
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

In [ ]:
df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)
#
#"instruct_data" : le chemin vers le fichier de données d'entraînement (ici, un fichier JSONL qui contient des exemples d'instructions ou de dialogues pour entraîner le modèle).

#"eval_instruct_data" : le chemin vers le fichier de données d'évaluation (souvent utilisé pour mesurer la performance du modèle sur des exemples non vus pendant l'entraînement).

In [ ]:
data = {
  "instruct_data": "/Users/johndoe/data/ultrachat_chunk_train.jsonl",
  "eval_instruct_data": "/Users/johndoe/data/ultrachat_chunk_eval.jsonl"
}
eval_instruct_data: "/Users/johndoe/data/ultrachat_chunk_eval.jsonl"

Cela permet d'avoir du coup d'un côté , l'ensemble d'entraînementqui est créé au format JSON.
Et de test au format JSOn.

In [ ]:
import os
import json

# Chemins vers vos fichiers JSONL
train_file = "/Users/johndoe/data/ultrachat_chunk_train.jsonl"
eval_file = "/Users/johndoe/data/ultrachat_chunk_eval.jsonl"

# Création des dossiers parents si nécessaire
os.makedirs(os.path.dirname(train_file), exist_ok=True)
os.makedirs(os.path.dirname(eval_file), exist_ok=True)

# Exemple de données d'entraînement
train_data = [
    {"text": "Instruction 1: Explique comment fonctionne LoRA et son utilité."},
    {"text": "Instruction 2: Décris les avantages de la quantification en 4 bits."},
    {"text": "Instruction 3: Donne un exemple d'utilisation de Mistral dans un contexte réel."}
]

# Exemple de données d'évaluation
eval_data = [
    {"text": "Évaluation 1: Quel est le rôle des adaptateurs dans LoRA ?"},
    {"text": "Évaluation 2: Comment la quantification en 4 bits affecte-t-elle la performance du modèle ?"}
]

# Écriture des données au format JSONL
with open(train_file, "w", encoding="utf-8") as f_train:
    for entry in train_data:
        f_train.write(json.dumps(entry) + "\n")

with open(eval_file, "w", encoding="utf-8") as f_eval:
    for entry in eval_data:
        f_eval.write(json.dumps(entry) + "\n")

print("Fichiers JSONL créés avec succès !")


Fichiers JSONL créés avec succès !


2eme essai fine tuning avec le Low Rank : cela permet notamment de diminuer le nombre de paramètres !

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset

# ------------------------------------------------------------
# Paramètres et chemins
# ------------------------------------------------------------
# Nom du modèle de base sur Hugging Face
base_model_name =  kagglehub.model_download('mistral-ai/mistral/PyTorch/7b-v0.1-hf/1')


# Chemins vers vos fichiers JSONL (données d'entraînement et d'évaluation)
train_file = "/Users/johndoe/data/ultrachat_chunk_train.jsonl"
eval_file = "/Users/johndoe/data/ultrachat_chunk_eval.jsonl"

# ------------------------------------------------------------
# Chargement du tokenizer
# ------------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# ------------------------------------------------------------
# Configuration BitsAndBytes pour le chargement en 4‑bits
# ------------------------------------------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,    # Utilisation de float16 pour le calcul
    bnb_4bit_quant_type="nf4",                 # Type de quantification (peut être "fp4" ou "nf4")
    bnb_4bit_use_double_quant=True             # Améliore la précision de la quantification
)

# ------------------------------------------------------------
# Chargement du modèle en 4‑bits
# ------------------------------------------------------------
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# ------------------------------------------------------------
# Configuration de LoRA
# On ne va entraîner que certaines parties (par exemple, les modules "q_proj" et "v_proj")
# Ce qui représente environ 1 à 2 % des paramètres du modèle.
# ------------------------------------------------------------
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                  # Rang de la décomposition
    lora_alpha=32,        # Facteur d'échelle
    lora_dropout=0.1,     # Dropout pour régulariser
    target_modules=["q_proj", "v_proj"]  # Modules à adapter (à ajuster selon le modèle)
)
model = get_peft_model(model, lora_config)

# ------------------------------------------------------------
# Chargement et préparation du dataset
# ------------------------------------------------------------
# Chargement des données JSONL avec le champ "text"
dataset = load_dataset("json", data_files={"train": train_file, "eval": eval_file})

def preprocess_function(examples):
    # Tokenisation du champ "text" avec troncature et longueur maximale définie
    return tokenizer(examples["text"], truncation=True, max_length=1024)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# ------------------------------------------------------------
# Configuration des arguments d'entraînement
# ------------------------------------------------------------
training_args = TrainingArguments(
    output_dir="./mistral_finetuned",       # Répertoire pour sauvegarder le modèle fine-tuné
    per_device_train_batch_size=1,          # Batch size faible pour limiter la mémoire
    per_device_eval_batch_size=1,
    num_train_epochs=3,                     # Nombre d'époques (à adapter)
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,                              # Utilisation du mode FP16 pour accélérer et réduire la mémoire
    gradient_accumulation_steps=8,          # Accumulation de gradients pour simuler un plus grand batch
    report_to="none"
)

# ------------------------------------------------------------
# Initialisation du Trainer et lancement de l'entraînement
# ------------------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"]
)

# Lancement de l'entraînement
trainer.train()

# Sauvegarde du modèle fine-tuné
model.save_pretrained("./mistral_finetuned")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

Le but du fine tuning est de surentraîner sur des données spécifiques.
On va lui apprendre à sortir du JSON et avec le moins de variation possible, sous un format spécifique et lin'tégrer dans un LLM par un APPEL DE FONCTION !
Les chatbots sont un fine tuning d'un LLM, on a un mot de plus...
Un chatbot : c'est une question et le modèle répond : c'est un skill la discussion.
Un fine tuning classique, c'est lui apprendre d'avoir une discsussion sous forme de Chat bot !
1:PROMPTING : c'est beaucoup moins cher et complexe !
Si probleme de contexte, donc c'est du fine tuning !

Le fine tuning apporte une sortie beaucoup plus précise alors que le prompt permet d'être plus précis en entrée.
De plus, cela fait des économies de Tokens, en lui apprenant un format spécifique et en le surentraînant.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
import kagglehub

# ------------------------------------------------------------
# Parameters and paths
# ------------------------------------------------------------
# Download the model if it doesn't exist locally
base_model_name_hf = "mistral-ai/mistral/PyTorch/7b-v0.1-hf/1"  # Hugging Face model ID
base_model_path = kagglehub.model_download(base_model_name_hf)  # Download from KaggleHub

train_file = "/Users/johndoe/data/ultrachat_chunk_train.jsonl"
eval_file = "/Users/johndoe/data/ultrachat_chunk_eval.jsonl"

# ------------------------------------------------------------
# Load the tokenizer
# ------------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# ------------------------------------------------------------
# BitsAndBytes configuration for 4-bit loading
# ------------------------------------------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",  # Options: "nf4" or "fp4"
    bnb_4bit_use_double_quant=True
)

# ------------------------------------------------------------
# Load the model in 4-bit
# ------------------------------------------------------------
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,  # Use the local path
    quantization_config=bnb_config,
    device_map="auto"
)


# ------------------------------------------------------------
# Configuration de LoRA
# Seules certaines parties (ici : "q_proj" et "v_proj") seront entraînées.
# ------------------------------------------------------------
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                  # Rang de la décomposition
    lora_alpha=32,        # Facteur d'échelle
    lora_dropout=0.1,     # Taux de dropout pour régulariser
    target_modules=["q_proj", "v_proj"]  # Adapter ces modules selon l'architecture du modèle
)
model = get_peft_model(model, lora_config)

# ------------------------------------------------------------
# Chargement et préparation du dataset
# ------------------------------------------------------------
# Les fichiers JSONL doivent contenir un champ "text"
dataset = load_dataset("json", data_files={"train": train_file, "eval": eval_file})

def preprocess_function(examples):
    tokenized = tokenizer(examples["text"], truncation=True, max_length=1024)
    # Ajout du champ "labels" identique aux "input_ids" pour le calcul de la perte
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# ------------------------------------------------------------
# Configuration des arguments d'entraînement
# ------------------------------------------------------------
training_args = TrainingArguments(
    output_dir="./mistral_finetuned",       # Répertoire de sauvegarde du modèle fine-tuné
    per_device_train_batch_size=1,          # Batch size faible pour limiter l'utilisation de la mémoire
    per_device_eval_batch_size=1,
    num_train_epochs=3,                     # Nombre d'époques (à adapter selon vos besoins)
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,                              # Utilisation du mode FP16 pour réduire la mémoire et accélérer
    gradient_accumulation_steps=8,          # Accumulation des gradients pour simuler un plus gros batch
    report_to="none"
)

# ------------------------------------------------------------
# Initialisation du Trainer en spécifiant label_names
# ------------------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    label_names=["labels"]   # Permet au Trainer d'identifier correctement le champ des labels
)

# ------------------------------------------------------------
# Lancement de l'entraînement et sauvegarde du modèle
# ------------------------------------------------------------
trainer.train()
model.save_pretrained("./mistral_finetuned")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TypeError: Trainer.__init__() got an unexpected keyword argument 'label_names'

Grosso modo, cette étape de fine tuning est en 2 éapes ⁉
1.On entraîne avec Lora sur une petite partie des parametres
2. On utilise au final un adaptateur qui permet alors d'adapter le reste des hyperparametres.
Cela utilise beaucoup moins de GPU.

2. RAG : base de données avec des morceaux de données pertinentes, et on va l'injecter dans le prompt !
Cela permet d'éviter de le réentraîner !


In [ ]:
!git clone https://github.com/h2oai/h2o-llm-studio.git
%cd h2o-llm-studio
!pip install -r requirements.txt


Cloning into 'h2o-llm-studio'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'h2o-llm-studio'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!python server.py


python3: can't open file '/content/server.py': [Errno 2] No such file or directory


https://github.com/mistralai/mistral-finetune

2.RAG


https://www.youtube.com/watch?v=kgWXbNVQnDw

In [ ]:
!pip install datasets
!pip install transformers faiss-cpu
# Optionnel : Installer les dépendances nécessaires (à exécuter dans une cellule Colab)
!pip install transformers faiss-cpu

import torch
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

from transformers import AutoModelForCausalLM

# Exemple d'identifiant corrigé (vérifiez sur Hugging Face Hub l'identifiant exact)
model_id = "mistral-ai/mistral-7b-v0.1-hf"

# Chargement du modèle en précisant éventuellement le type de repo
model = AutoModelForCausalLM.from_pretrained(model_id, repo_type="model")


# Chargement du tokenizer pour RAG
tokenizer = RagTokenizer.from_pretrained(model_name)

# Chargement du retriever avec un index factice pour tester (en production, remplacez par votre propre index)
retriever = RagRetriever.from_pretrained(
    model_name,
    index_name="exact",         # "exact" utilise un index standard
    use_dummy_dataset=True      # Utilise un dataset factice pour le test
)

# Chargement du modèle RAG en mode séquence (RagSequenceForGeneration)
model = RagSequenceForGeneration.from_pretrained(model_name, retriever=retriever)

# Exemple de requête
query = "What is the capital of France?"
input_dict = tokenizer(query, return_tensors="pt")

# Génération de la réponse
outputs = model.generate(input_ids=input_dict["input_ids"])
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("Question:", query)
print("Réponse générée :", result[0])


OSError: mistral-ai/mistral-7b-v0.1-hf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_wandb = user_secrets.get_secret("wandb")

ModuleNotFoundError: No module named 'kaggle_secrets'

In [ ]:
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine tuning mistral 7B',
    job_type="training",
    anonymous="allow"
)

NameError: name 'wandb' is not defined

3.Parmaeter les hyperparametres

In [ ]:
!pip install transformers==4.29.0

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Chargement du dataset IMDb
dataset = load_dataset("imdb")

# Chargement du tokenizer et du modèle (ici, BERT pour la classification)
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Préparation du dataset : tokenisation et renommage de la colonne "label" en "labels"
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Définition des TrainingArguments
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

# Création du Trainer
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Lancement de l'entraînement
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

ValueError: paged_adamw_32bit is not a valid OptimizerNames, please select one of ['adamw_hf', 'adamw_torch', 'adamw_torch_fused', 'adamw_torch_xla', 'adamw_apex_fused', 'adafactor', 'adamw_bnb_8bit', 'adamw_anyprecision', 'sgd', 'adagrad']

output_dir="./results"
Répertoire dans lequel les checkpoints, les modèles sauvegardés et les logs seront stockés pendant et après l'entraînement.

num_train_epochs=1
Nombre d'époques d'entraînement. Ici, le modèle passera une fois sur l'ensemble des données d'entraînement.

per_device_train_batch_size=4
Taille du batch (lot) pour chaque appareil (par exemple, GPU) utilisé pour l'entraînement. Cela signifie que chaque GPU traitera 4 exemples à la fois.

gradient_accumulation_steps=1
Nombre de pas pour accumuler les gradients avant de faire une mise à jour des poids. Avec une valeur de 1, la mise à jour se fait à chaque batch. Une valeur supérieure permet de simuler un batch de taille plus importante sans avoir besoin de beaucoup de mémoire.

optim="paged_adamw_32bit"
Spécifie l'optimiseur utilisé pour l'entraînement. Ici, il s'agit d'une variante d'AdamW adaptée pour la gestion efficace de la mémoire en 32 bits (souvent utilisée pour les grands modèles).

save_steps=25
Le modèle est sauvegardé toutes les 25 étapes (iterations). Cela permet de créer des points de contrôle réguliers pour pouvoir reprendre l'entraînement en cas d'interruption.

logging_steps=25
Les informations de logging (comme la perte, la métrique, etc.) sont affichées ou enregistrées toutes les 25 étapes.

learning_rate=2e-4
Le taux d'apprentissage définit la vitesse à laquelle le modèle ajuste ses poids. Ici, il est fixé à 0,0002.

weight_decay=0.001
Le coefficient de décroissance des poids, qui aide à régulariser le modèle en pénalisant les poids trop élevés, contribuant ainsi à éviter le surapprentissage (overfitting).

fp16=False
Désactive l'utilisation de la précision mixte en 16 bits (FP16). Cela peut être utile pour accélérer l'entraînement sur certains matériels, mais ici, on le désactive.

bf16=False
Désactive l'utilisation du format bfloat16, une autre précision mixte qui peut améliorer la vitesse et réduire l'utilisation mémoire sur certains matériels (généralement sur TPU).

max_grad_norm=0.3
La norme maximale pour le clipping des gradients. Ce paramètre aide à stabiliser l'entraînement en limitant la valeur des gradients et ainsi prévenir les mises à jour trop importantes qui pourraient déstabiliser l'apprentissage.

max_steps=-1
Un nombre négatif indique qu'il n'y a pas de limite explicite sur le nombre total de pas d'entraînement. L'entraînement s'arrêtera donc uniquement en fonction du nombre d'époques spécifié.

warmup_ratio=0.03
Le ratio de pas de warm-up par rapport au nombre total de pas. Durant la phase de warm-up, le taux d'apprentissage augmente progressivement pour atteindre la valeur définie, ce qui peut aider à stabiliser l'entraînement dès le début.

group_by_length=True
Lorsque cette option est activée, les exemples sont regroupés par longueur similaire. Cela permet d'optimiser l'utilisation de la mémoire en réduisant la quantité de padding nécessaire dans les batches.

lr_scheduler_type="constant"
Type de planificateur de taux d'apprentissage. Ici, il s'agit d'un scheduler constant, c'est-à-dire que le taux d'apprentissage reste fixe pendant l'entraînement (hormis éventuellement la phase de warm-up).

report_to="wandb"
Indique que les logs et métriques seront envoyés à Weights & Biases (wandb), un outil de suivi d'expériences d'entraînement. Cela permet de visualiser les courbes d'apprentissage, la perte, et d'autres métriques en temps réel sur le tableau de bord wandb.

Cette étape, qui consiste à définir les TrainingArguments, permet de configurer entièrement l'environnement et les paramètres d'entraînement du modèle. Plus précisément, elle permet de :

Définir le lieu et la fréquence de sauvegarde :
Le modèle est sauvegardé dans le dossier spécifié (output_dir) toutes les 25 étapes (via save_steps), ce qui permet de conserver des points de contrôle réguliers.

Contrôler la durée et le rythme de l'entraînement :
On précise le nombre d'époques (num_train_epochs), la taille des lots par appareil (per_device_train_batch_size) et la stratégie d'accumulation des gradients (gradient_accumulation_steps). Cela permet d'ajuster l'entraînement en fonction des ressources disponibles.

Définir l'optimiseur et les hyperparamètres associés :
Le choix de l'optimiseur (optim), le taux d'apprentissage (learning_rate), le weight decay, et le clipping des gradients (max_grad_norm) influencent la manière dont le modèle ajuste ses poids.

Gérer la planification du taux d'apprentissage :
Le warmup_ratio permet de démarrer l'entraînement de manière progressive, et le lr_scheduler_type ("constant" dans cet exemple) définit comment le taux d'apprentissage évolue durant l'entraînement.

Optimiser l'utilisation des ressources :
Des options comme group_by_length aident à optimiser l'utilisation de la mémoire en regroupant des séquences de longueur similaire, ce qui réduit le padding inutile.

Configurer le reporting des métriques :
En indiquant report_to="wandb", les logs et métriques de l'entraînement seront envoyés à Weights & Biases, facilitant ainsi le suivi et l'analyse de l'entraînement en temps réel.

En résumé, cette étape permet de spécifier tous les aspects liés au déroulement de l'entraînement, garantissant ainsi que le processus se réalise de manière contrôlée, efficace et traçable.








In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

NameError: name 'SFTTrainer' is not defined

4.Bibliotheque SFTTtraiing avec peu de code

In [ ]:
# Installer les dépendances nécessaires
!pip install transformers datasets trl peft

import os
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig
from trl import SFTTrainer, SFTTrainingArguments

# ------------------------------------------------------------
# 1. Création de données d'exemple en JSONL (si non existantes)
# ------------------------------------------------------------
train_file = "train_data.jsonl"
val_file = "val_data.jsonl"

if not os.path.exists(train_file):
    with open(train_file, "w", encoding="utf-8") as f:
        f.write(json.dumps({"text": "Ceci est un exemple d'instruction pour l'entraînement."}) + "\n")
        f.write(json.dumps({"text": "Un autre exemple d'instruction pour améliorer le modèle."}) + "\n")

if not os.path.exists(val_file):
    with open(val_file, "w", encoding="utf-8") as f:
        f.write(json.dumps({"text": "Ceci est un exemple d'instruction pour la validation."}) + "\n")

# ------------------------------------------------------------
# 2. Chargement du dataset
# ------------------------------------------------------------
dataset = load_dataset("json", data_files={"train": train_file, "validation": val_file})

# ------------------------------------------------------------
# 3. Chargement du modèle et du tokenizer
# Ici, nous utilisons GPT-2 pour l'exemple.
# ------------------------------------------------------------
model_name = "gpt2"  # Vous pouvez remplacer par le modèle de votre choix
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# ------------------------------------------------------------
# 4. Configuration de LoRA (Low-Rank Adaptation)
# Cette configuration permet d'entraîner seulement une partie des paramètres.
# ------------------------------------------------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],  # Module ciblé pour GPT-2 (à adapter selon le modèle)
    lora_dropout=0.1,
    bias="none"
)

# ------------------------------------------------------------
# 5. Configuration des paramètres d'entraînement via SFTTrainingArguments
# ------------------------------------------------------------
training_args = SFTTrainingArguments(
    output_dir="./sft_results",            # Répertoire de sauvegarde
    num_train_epochs=3,                    # Nombre d'époques d'entraînement
    per_device_train_batch_size=2,         # Taille de batch pour l'entraînement
    per_device_eval_batch_size=2,          # Taille de batch pour la validation
    logging_steps=10,                      # Fréquence d'affichage des logs
    save_steps=50,                         # Fréquence de sauvegarde du modèle
    learning_rate=5e-5,                    # Taux d'apprentissage
    fp16=True                              # Utilisation de la précision mixte (FP16) si possible
)

# ------------------------------------------------------------
# 6. Création du SFT Trainer en fournissant tous les composants
# ------------------------------------------------------------
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    args=training_args,
    lora_config=lora_config  # Ajoute la configuration LoRA pour un fine-tuning efficace
)

# ------------------------------------------------------------
# 7. Lancement de l'entraînement
# ------------------------------------------------------------
trainer.train()

# Optionnel : Sauvegarde finale du modèle fine-tuné
model.save_pretrained("./sft_results/final_model")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 5.1 MB/s eta 0:00:00


RuntimeError: Failed to import trl.trainer.sft_trainer because of the following error (look up to see its traceback):
Failed to import transformers.processing_utils because of the following error (look up to see its traceback):
cannot import name 'load_video' from 'transformers.image_utils' (/usr/local/lib/python3.11/dist-packages/transformers/image_utils.py)